In [3]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
import random
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

list_of_action = {
    0: (0,0),
    1: (0,1),
    2: (0,2),
    3: (1,0),
    4: (1,1),
    5: (1,2),
    6: (2,0),
    7: (2,1),
    8: (2,2)
}

rev_action = {
    (0,0) :0,
    (0,1) :1,
    (0,2) :2,
    (1,0) :3,
    (1,1) :4,
    (1,2) :5,
    (2,0) :6,
    (2,1) :7,
    (2,2) :8
}

In [10]:

def find_aval_actions(state_of_game,act):
  possible_actions = []
  for i in range(9):
    if state_of_game[act[i]] == 0:
      possible_actions.append(i)
  return possible_actions

In [5]:
class DQN(nn.Module):

    def __init__(self, in_features, outputs, action_list):
        super(DQN, self).__init__()
        
        self.action_list = action_list
        
        
        self.head1 = nn.Linear(in_features, 128)
        self.head2 = nn.Linear(128,outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        #print('input x', x.shape, type(x))
        x = F.relu(self.head1(x.view(x.size(0), -1))
        x = self.head2(x)
        #print('output x',x.shape)
        return x
    
    def act(self, obs):
        obs_tensor = torch.as_tensor(obs, dtype=torch.float32)
        #mask_possible_action = torch.ne(obs_tensor,0)*-1
        #print('mask_possible_action',mask_possible_action)
        
        q_values = self.forward(obs_tensor.unsqueeze(0))
        #possible_q_values = mask_possible_action*q_values
        #print('possible_q_values',possible_q_values)
        
        
        max_q_idx = torch.argmax(q_values, dim=1)[0]
        max_q_idx_item = max_q_idx.detach().item()
        #print('max_q_idx_item', max_q_idx_item, type(max_q_idx_item))
        #print('action list', self.action_list)
        #print('rev_action',rev_action)
        move = self.action_list[max_q_idx_item]
        
        action_idx = rev_action[move]
        
        return action_idx
    
        

In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE_FREQ = 500

number_of_actions = 9
grid_size = 9


env = TictactoeEnv()
replay_buffer = deque(maxlen=BUFFER_SIZE)
rew_buffer = deque([0.0], maxlen=10)

episode_reward = 0.0

#initialization network
policy_net = DQN(grid_size, number_of_actions,list_of_action).to("cpu")
target_net = DQN(grid_size, number_of_actions,list_of_action).to("cpu")
target_net.load_state_dict(policy_net.state_dict())
#target_net.eval()





<All keys matched successfully>

In [17]:
Turns = np.array(['X','O'])
MIN_REPLAY_SIZE = 5000

#Initialize the Replay Buffer
replay_buffer = deque(maxlen=BUFFER_SIZE)

game = 0
episode_reward = 0.0

while(len(replay_buffer) < MIN_REPLAY_SIZE):
    game += 1
    print('game', game, 'buffer', len(replay_buffer))
    env.reset()
    #env.render()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt = OptimalPlayer(epsilon=0.7, player='O')
    player_rnd = OptimalPlayer(epsilon=1., player='X')
    
    for j in range(5):
        #print('step', j)
       
        #random agent play
        obs = env.observe()[0].flatten()
        move = player_rnd.act(grid)
        grid, end, winner = env.step(move, print_grid=False)
        action = rev_action[move]
        
        if not end:
            #optimal player play
            move = player_opt.act(grid)
            grid, end, winner = env.step(move, print_grid=False)
        
        new_obs = env.observe()[0].flatten()
       
    
        #transition set-off
        rew = env.reward(player=Turns[1])
        transition = (obs, action, rew, end, new_obs)
        replay_buffer.append(transition)
           

        if end:
            #print('-------------------------------------------')
            #print('Game end, winner is player ' + str(winner))
            #print('Optimal player = ' +  Turns[0])
            #print('Random player = ' +  Turns[1])
            #env.render()
            env.reset()
            break




#for _ in range(MIN_REPLAY_SIZE):
    
   

game 1 buffer 0
game 2 buffer 3
game 3 buffer 7
game 4 buffer 11
game 5 buffer 15
game 6 buffer 18
game 7 buffer 23
game 8 buffer 26
game 9 buffer 31
game 10 buffer 36
game 11 buffer 40
game 12 buffer 43
game 13 buffer 47
game 14 buffer 50
game 15 buffer 53
game 16 buffer 57
game 17 buffer 61
game 18 buffer 64
game 19 buffer 68
game 20 buffer 72
game 21 buffer 75
game 22 buffer 78
game 23 buffer 83
game 24 buffer 88
game 25 buffer 92
game 26 buffer 96
game 27 buffer 100
game 28 buffer 103
game 29 buffer 107
game 30 buffer 110
game 31 buffer 114
game 32 buffer 118
game 33 buffer 122
game 34 buffer 126
game 35 buffer 130
game 36 buffer 135
game 37 buffer 140
game 38 buffer 145
game 39 buffer 149
game 40 buffer 153
game 41 buffer 156
game 42 buffer 160
game 43 buffer 163
game 44 buffer 166
game 45 buffer 170
game 46 buffer 174
game 47 buffer 177
game 48 buffer 181
game 49 buffer 184
game 50 buffer 187
game 51 buffer 191
game 52 buffer 195
game 53 buffer 199
game 54 buffer 202
game 55 buff

game 411 buffer 1633
game 412 buffer 1638
game 413 buffer 1642
game 414 buffer 1646
game 415 buffer 1650
game 416 buffer 1654
game 417 buffer 1658
game 418 buffer 1663
game 419 buffer 1668
game 420 buffer 1672
game 421 buffer 1676
game 422 buffer 1681
game 423 buffer 1684
game 424 buffer 1688
game 425 buffer 1693
game 426 buffer 1696
game 427 buffer 1699
game 428 buffer 1702
game 429 buffer 1707
game 430 buffer 1712
game 431 buffer 1717
game 432 buffer 1721
game 433 buffer 1726
game 434 buffer 1731
game 435 buffer 1734
game 436 buffer 1739
game 437 buffer 1743
game 438 buffer 1748
game 439 buffer 1752
game 440 buffer 1757
game 441 buffer 1761
game 442 buffer 1764
game 443 buffer 1769
game 444 buffer 1773
game 445 buffer 1778
game 446 buffer 1782
game 447 buffer 1787
game 448 buffer 1792
game 449 buffer 1796
game 450 buffer 1799
game 451 buffer 1803
game 452 buffer 1808
game 453 buffer 1813
game 454 buffer 1816
game 455 buffer 1820
game 456 buffer 1823
game 457 buffer 1827
game 458 buff

game 813 buffer 3258
game 814 buffer 3261
game 815 buffer 3264
game 816 buffer 3267
game 817 buffer 3272
game 818 buffer 3275
game 819 buffer 3278
game 820 buffer 3282
game 821 buffer 3287
game 822 buffer 3291
game 823 buffer 3295
game 824 buffer 3298
game 825 buffer 3303
game 826 buffer 3307
game 827 buffer 3312
game 828 buffer 3316
game 829 buffer 3321
game 830 buffer 3325
game 831 buffer 3329
game 832 buffer 3332
game 833 buffer 3335
game 834 buffer 3340
game 835 buffer 3344
game 836 buffer 3349
game 837 buffer 3352
game 838 buffer 3356
game 839 buffer 3360
game 840 buffer 3364
game 841 buffer 3368
game 842 buffer 3372
game 843 buffer 3376
game 844 buffer 3379
game 845 buffer 3383
game 846 buffer 3388
game 847 buffer 3391
game 848 buffer 3396
game 849 buffer 3399
game 850 buffer 3403
game 851 buffer 3408
game 852 buffer 3411
game 853 buffer 3414
game 854 buffer 3417
game 855 buffer 3421
game 856 buffer 3426
game 857 buffer 3431
game 858 buffer 3435
game 859 buffer 3439
game 860 buff

game 1207 buffer 4839
game 1208 buffer 4843
game 1209 buffer 4848
game 1210 buffer 4853
game 1211 buffer 4856
game 1212 buffer 4859
game 1213 buffer 4863
game 1214 buffer 4866
game 1215 buffer 4871
game 1216 buffer 4875
game 1217 buffer 4880
game 1218 buffer 4884
game 1219 buffer 4889
game 1220 buffer 4892
game 1221 buffer 4896
game 1222 buffer 4899
game 1223 buffer 4902
game 1224 buffer 4906
game 1225 buffer 4911
game 1226 buffer 4916
game 1227 buffer 4921
game 1228 buffer 4926
game 1229 buffer 4931
game 1230 buffer 4934
game 1231 buffer 4939
game 1232 buffer 4942
game 1233 buffer 4945
game 1234 buffer 4949
game 1235 buffer 4953
game 1236 buffer 4957
game 1237 buffer 4962
game 1238 buffer 4965
game 1239 buffer 4969
game 1240 buffer 4972
game 1241 buffer 4976
game 1242 buffer 4981
game 1243 buffer 4985
game 1244 buffer 4988
game 1245 buffer 4993
game 1246 buffer 4996


In [18]:
print(len(replay_buffer))

5000


In [26]:
#Main Training Loop

env.reset()
env.render()
player_opt = OptimalPlayer(epsilon=0.7, player='O')

optimizer = torch.optim.Adam(policy_net.parameters(), lr=5e-4)
rew_buffer = deque([0.0], maxlen=50)
bad_pred_buffer = deque([0.0], maxlen=50)
random_buffer = deque([0.0], maxlen=50)

game = 0
old_game = 0
step = 0
bad_prediction = False

episode_reward = 0.0
episode_bad_predict = 0.0
episode_random = 0.0

criterion = torch.nn.HuberLoss(delta=1.0)
while(game < 1000):
    step +=1
    epsilon = np.interp(step, [0, EPS_DECAY], [EPS_START,EPS_END])
    
    rnd_sample = random.random()
    #print('rnd_sample', rnd_sample)
    
   
    #random agent play
    obs = env.observe()[0]
    available_action = find_aval_actions(obs, list_of_action)
    if rnd_sample <= epsilon:
        random_int = random.randint(0,len(available_action)-1)
        #print('random_int', random_int, 'len available action',len(available_action))
        move = available_action[random_int]
        episode_random +=1
    else:
        move = policy_net.act(obs)
    
    
    
    if not move in available_action:
        #print(' bad prediction', available_action)
        bad_prediction = True
        
    obs = env.observe()[0].flatten()
    #print('move prediction', move)
    #env.render()
    #print('move', move)
    
    if not bad_prediction:
        #print('env step')
        grid, end, winner = env.step(move, print_grid=False)
        #env.render()
        
    action = move
    #print('###### action type #####',type(action))
    #env.render()
    
    if (not end) and (not bad_prediction):
        #optimal player play
        move = player_opt.act(grid)
        #print('optimal move', move)
        
        grid, end, winner = env.step(move, print_grid=False)
        #env.render()
    

    #transition set-off
    new_obs = env.observe()[0].flatten()
    if not bad_prediction:
        rew = env.reward(player='X')
    else:
        rew = -1
    transition = (obs, action, rew, end, new_obs)
    #print('transition', transition)
    replay_buffer.append(transition)
    
    
    if end or bad_prediction:
        #print('game finish')
        if bad_prediction:
            episode_bad_predict += 1
            
        bad_prediction = False
        #print('-------------------------------------------')
        #print('Game end, winner is player ' + str(winner))
        #print('Optimal player = ' +  Turns[0])
        #print('Random player = ' +  Turns[1])
        #env.render()
        env.reset()
        episode_reward += rew
        game += 1
        if game % 10 == 0:
            
            rew_buffer.append(episode_reward)
            bad_pred_buffer.append(episode_bad_predict)
            random_buffer.append(episode_random)

            # Logging
            if game % 10 == 0:
                print()
                print('game', game)
                print('episode random',episode_random)
                print('Avg random', np.mean(random_buffer))
                print('episode bad predi',episode_bad_predict)
                print('Avg bad predi', np.mean(bad_pred_buffer))
                print('episode rew',episode_reward)
                print('Avg Rew', np.mean(rew_buffer)) 
                print('Loss', loss)
            episode_reward = 0.0
            episode_bad_predict = 0.0
            episode_random = 0.0
    
    
    # Start Gradient Step
    #print('replay_buffer', len(replay_buffer))
    transitions = random.sample(replay_buffer, BATCH_SIZE)
    
    obses = np.asarray([t[0] for t in transitions])
    #print('obsas', type(obses))
    actions = np.asarray([t[1] for t in transitions])
    #print('actions type', type(actions))
    #print(actions)
    rews = np.asarray([t[2] for t in transitions])
    dones = np.asarray([t[3] for t in transitions])
    new_obses = np.asarray([t[4] for t in transitions])
    
    obses_tensor = torch.as_tensor(obses, dtype=torch.float32)
    actions_tensor = torch.as_tensor(actions, dtype=torch.int64)#.unsqueeze(-1)
    rews_tensor = torch.as_tensor(rews, dtype=torch.float32).unsqueeze(-1)
    dones_tensor = torch.as_tensor(dones, dtype=torch.float32).unsqueeze(-1)
    #print('dones tensor', dones_tensor.shape)
    new_obses_tensor = torch.as_tensor(new_obses, dtype=torch.float32)
    
    # Compute Targets
    target_q_values = target_net(new_obses_tensor)
    #print('target_q_values',target_q_values.shape)
    mask_possible_action = torch.ne(new_obses_tensor,0)*-1
    #print('mask_possible_action',mask_possible_action.shape)
    possible_target_q_values = target_q_values * mask_possible_action
    #print('possible_target_q_values',possible_target_q_values.shape)
    max_target_q_values = possible_target_q_values.max(dim=1, keepdim=True)[0]
    #print('max_target_q_values', max_target_q_values.shape)
    
    targets = rews_tensor + GAMMA * (1 - dones_tensor) * max_target_q_values
    #print('targets', targets.shape)
    
    
    # Compute Loss
    
    q_values = policy_net(obses_tensor)
    q_values_max = q_values.max(dim=1, keepdim=True)[0]
    
    
    #print('q_values',q_values.shape,'q_values_max',q_values_max.shape)
    
    #action_q_values = torch.gather(input=q_values, dim=1, index=actions_tensor)
    
    #print('action_q_values',action_q_values.shape, 'targets',targets.shape)
    
    #loss = nn.functional.smooth_l1_loss(q_values_max, targets)
    loss = criterion(q_values_max, targets)
   
    
    #print('loss', loss, loss.shape, type(loss))
#     max_q_idx = torch.argmax(possible_q_values, dim=1)[0]
#     max_q_idx_item = max_q_idx.detach().item()
    
    
    # Gradient Descent
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Update Target Network
    if step % TARGET_UPDATE_FREQ == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
    # Logging
    


|- - -|
|- - -|
|- - -|


game 10
episode random 32.0
Avg random 16.0
episode bad predi 4.0
Avg bad predi 2.0
episode rew -2.0
Avg Rew -1.0
Loss tensor(0.1259, grad_fn=<HuberLossBackward0>)

game 20
episode random 21.0
Avg random 17.666666666666668
episode bad predi 5.0
Avg bad predi 3.0
episode rew -8.0
Avg Rew -3.3333333333333335
Loss tensor(0.1918, grad_fn=<HuberLossBackward0>)

game 30
episode random 18.0
Avg random 17.75
episode bad predi 7.0
Avg bad predi 4.0
episode rew -8.0
Avg Rew -4.5
Loss tensor(0.1475, grad_fn=<HuberLossBackward0>)

game 40
episode random 11.0
Avg random 16.4
episode bad predi 8.0
Avg bad predi 4.8
episode rew -8.0
Avg Rew -5.2
Loss tensor(0.1620, grad_fn=<HuberLossBackward0>)

game 50
episode random 7.0
Avg random 14.833333333333334
episode bad predi 8.0
Avg bad predi 5.333333333333333
episode rew -10.0
Avg Rew -6.0
Loss tensor(0.2566, grad_fn=<HuberLossBackward0>)

game 60
episode random 1.0
Avg random 12.857142857142858
episode bad predi 10.0
Avg bad pre


game 450
episode random 2.0
Avg random 3.260869565217391
episode bad predi 8.0
Avg bad predi 8.565217391304348
episode rew -6.0
Avg Rew -8.521739130434783
Loss tensor(0.1986, grad_fn=<HuberLossBackward0>)

game 460
episode random 0.0
Avg random 3.1914893617021276
episode bad predi 10.0
Avg bad predi 8.595744680851064
episode rew -10.0
Avg Rew -8.553191489361701
Loss tensor(0.1540, grad_fn=<HuberLossBackward0>)

game 470
episode random 2.0
Avg random 3.1666666666666665
episode bad predi 9.0
Avg bad predi 8.604166666666666
episode rew -8.0
Avg Rew -8.541666666666666
Loss tensor(0.1679, grad_fn=<HuberLossBackward0>)

game 480
episode random 3.0
Avg random 3.163265306122449
episode bad predi 7.0
Avg bad predi 8.571428571428571
episode rew -6.0
Avg Rew -8.489795918367347
Loss tensor(0.1633, grad_fn=<HuberLossBackward0>)

game 490
episode random 3.0
Avg random 3.16
episode bad predi 9.0
Avg bad predi 8.58
episode rew -8.0
Avg Rew -8.48
Loss tensor(0.1247, grad_fn=<HuberLossBackward0>)

game

Loss tensor(0.2508, grad_fn=<HuberLossBackward0>)

game 940
episode random 0.0
Avg random 1.8
episode bad predi 7.0
Avg bad predi 8.54
episode rew -10.0
Avg Rew -8.22
Loss tensor(0.1757, grad_fn=<HuberLossBackward0>)

game 950
episode random 1.0
Avg random 1.78
episode bad predi 6.0
Avg bad predi 8.5
episode rew -6.0
Avg Rew -8.22
Loss tensor(0.1283, grad_fn=<HuberLossBackward0>)

game 960
episode random 2.0
Avg random 1.82
episode bad predi 9.0
Avg bad predi 8.48
episode rew -10.0
Avg Rew -8.22
Loss tensor(0.2271, grad_fn=<HuberLossBackward0>)

game 970
episode random 3.0
Avg random 1.84
episode bad predi 9.0
Avg bad predi 8.48
episode rew -10.0
Avg Rew -8.26
Loss tensor(0.1657, grad_fn=<HuberLossBackward0>)

game 980
episode random 0.0
Avg random 1.78
episode bad predi 9.0
Avg bad predi 8.52
episode rew -10.0
Avg Rew -8.34
Loss tensor(0.1699, grad_fn=<HuberLossBackward0>)

game 990
episode random 1.0
Avg random 1.74
episode bad predi 10.0
Avg bad predi 8.54
episode rew -10.0
Avg Rew 